In [1]:
import torch
import os
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
from torch.utils.data import Dataset,DataLoader
import gc
from mpl_toolkits.mplot3d import Axes3D
print('Package import Completed!')

Package import Completed!


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
filePath='OULA.DataSet/'
df_stuVle=pd.read_csv(filePath + 'studentVle.csv')
df_Vle=pd.read_csv(filePath + 'Vle.csv')
df_Courses=pd.read_csv(filePath + 'courses.csv')
df_Stu=pd.read_csv(filePath + 'studentInfo.csv')
df_stuVle=pd.merge(df_stuVle,df_Vle,on='id_site')
df_stuVle=pd.merge(df_stuVle,df_Stu,left_on=['id_student','code_module_x','code_presentation_x'],right_on=['id_student','code_module','code_presentation'])

def clicks_norm(df):
    min_value = df['sum_click'].min()
    max_value = df['sum_click'].max()
    df['sum_click'] = df['sum_click'].apply(
        lambda x:(x)/(max_value))
    return df

In [10]:
test_percent=0.25
batch_size=200
learnrate=0.001
countEpoch=300
Code_Size=48
num_Course=22

lossList=[]
res=[]

course_ID=0;
for index,course in df_Courses.iterrows():
    strThisCourse = course['code_module']+'_'+course['code_presentation']
    print(strThisCourse + ' Start...')
    thisCourseLoss=[]
    thisCourseLoss.append(strThisCourse)
    df_CurrentCourse=df_stuVle[(df_stuVle['code_module']==course['code_module']) & (df_stuVle['code_presentation']==course['code_presentation'])]
    df_CurrentCourse=df_CurrentCourse.loc[:,['id_student','date','activity_type','sum_click','final_result']]
       
    keyActitities=['externalquiz',
                   'forumng',
                   'glossary',
                   'oucollaborate',
                   'oucontent',
                   'ouelluminate',
                   'ouwiki',
                   'questionnaire',
                   'quiz',
                   'resource']
#                    'dataplus',
#                    'folder',
#                    'dualpane',
#                    'sharedsubpage',
#                    'homepage',
#                    'subpage',
#                    'page',
#                    'url',
#                    'htmlactivity',
#                    'repeatactivity'
#                    ]
    df_CurrentCourse=df_CurrentCourse[df_CurrentCourse['activity_type'].isin(keyActitities)]
    dates=df_CurrentCourse['date'].unique()
    dates=np.sort(dates)
    df_CurrentCourse = df_CurrentCourse.groupby(['activity_type']).apply(clicks_norm)
    
    listCurrentStu=[] 
    listFinalRes=[] 
    finalRes='None'
    for id in df_CurrentCourse['id_student'].unique():
        df_id_vles=df_CurrentCourse[df_CurrentCourse['id_student']==id] 
        stu_dates=dict.fromkeys(dates,None)
        for ele in stu_dates.keys():
            stu_dates[ele]=dict.fromkeys(keyActitities,0)
        for index,row in df_id_vles.iterrows():
            (stu_dates[row['date']])[row['activity_type']]=row['sum_click']
            finalRes=row['final_result']  
        this_student=[]
        for oneday in stu_dates.keys():
            this_student.append(list(stu_dates[oneday].values()))
        listCurrentStu.append(this_student)
        listFinalRes.append(finalRes)
    
    ALLDataSet=np.array(listCurrentStu) 
    ALLDataSet=torch.from_numpy(ALLDataSet) 
    ALLDataSet=ALLDataSet.to(torch.float32).to(device) 
    print('ALLDataSet',ALLDataSet.size())
    
    countALLSamples = ALLDataSet.size()[0] 
    countSeq=ALLDataSet.size()[1] 
    countFeatures=ALLDataSet.size()[2] 
    ALLDataSet=ALLDataSet.reshape(-1,countSeq*countFeatures)
    
    class MData(Dataset):
        def __init__(self,x):
            self.data=x
            self._x=x
        def __getitem__(self,index):
            return self._x[index]
        def __len__(self):
            return len(self.data)
    
    AEData=MData(ALLDataSet)
    AEDataLoader=DataLoader(AEData,batch_size=batch_size,shuffle=False,drop_last=False,num_workers=0)

    class MyAE(nn.Module):
        def __init__(self,input_dim,AE_code_size):
            super(MyAE, self).__init__()
            self._dims=input_dim
            self.Encoder=nn.Sequential(
                nn.Linear(self._dims, self._dims),
                nn.ReLU(),
                nn.Linear(self._dims, 2048),
                nn.ReLU(),
                nn.Linear(2048, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, AE_code_size),
            )
            self.Decoder=nn.Sequential(
                nn.Linear(AE_code_size, 256),
                nn.ReLU(),
                nn.Linear(256, 512),
                nn.ReLU(),
                nn.Linear(512, 2048),
                nn.ReLU(),
                nn.Linear(2048, self._dims),
            )
            self.criterion = nn.CrossEntropyLoss()

        def forward(self,x):
            en = self.Encoder(x)
            de = self.Decoder(en)
            return en,de
    
    encoder=(MyAE(countSeq*countFeatures,Code_Size)).to(device)    
    
    optimizer = torch.optim.Adam(encoder.parameters(),lr=learnrate)
    gc.collect()
    torch.cuda.empty_cache()
    for epoch in range(countEpoch):
        for i,data in enumerate(AEDataLoader):
            x=data
            en,de=encoder(x)
            loss=encoder.criterion(de,x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            gc.collect()
            torch.cuda.empty_cache()
        with torch.no_grad():
            en,de = encoder(ALLDataSet)
            allloss=encoder.criterion(de,ALLDataSet)
    
    en,de = encoder(ALLDataSet)
    
    filepath='Enc\\'+strThisCourse+'.'+str(Code_Size)+'.Enc.csv'
    df_2D=pd.DataFrame(en.detach().cpu().numpy()) 
    df_2D['label']=listFinalRes
    df_2D.to_csv(filepath,index=False)
    
#     #----------------------------------------------------------------------------------------------------------------3D Drawing
#     df_2D=pd.read_csv(filepath)
#     fig=plt.figure()
#     alpha=0.5
#     size=8
#     ax = fig.gca(projection='3d')
#     ax.scatter(df_2D[df_2D['label']=='Pass']['0'],df_2D[df_2D['label']=='Pass']['1'],df_2D[df_2D['label']=='Pass']['2'],
#                c='g',label='Pass',s=size,
#                alpha=alpha)
#     ax.scatter(df_2D[df_2D['label']=='Withdrawn']['0'],df_2D[df_2D['label']=='Withdrawn']['1'],df_2D[df_2D['label']=='Withdrawn']['2'],
#                c='k',label='Withdrawn',s=size,
#                alpha=alpha)
#     ax.scatter(df_2D[df_2D['label']=='Fail']['0'],df_2D[df_2D['label']=='Fail']['1'],df_2D[df_2D['label']=='Fail']['2'],
#                c='r',label='Fail',s=5,
#                alpha=alpha)
#     ax.scatter(df_2D[df_2D['label']=='Distinction']['0'],df_2D[df_2D['label']=='Distinction']['1'],df_2D[df_2D['label']=='Distinction']['2'],
#                c='b',label='Distinction',s=size,
#                alpha=alpha)
#     ax.legend(loc='upper right')
#     #fig.label(strThisCourse)
#     plt.title(strThisCourse,loc='left')
#     plt.savefig('Figures.3D\\'+strThisCourse+'.svg',dpi=300,bbox_inches='tight')
#     plt.show()
    #----------------------------------------------------------------------------------------------------------------3D Drawing
    print(strThisCourse + ' Completed!')
    course_ID=course_ID+1

AAA_2013J Start...
ALLDataSet torch.Size([378, 279, 10])
AAA_2013J Completed!
AAA_2014J Start...
ALLDataSet torch.Size([354, 294, 10])
AAA_2014J Completed!
BBB_2013J Start...
ALLDataSet torch.Size([1836, 280, 10])
BBB_2013J Completed!
BBB_2014J Start...
ALLDataSet torch.Size([1908, 272, 10])
BBB_2014J Completed!
BBB_2013B Start...
ALLDataSet torch.Size([1521, 250, 10])
BBB_2013B Completed!
BBB_2014B Start...
ALLDataSet torch.Size([1269, 242, 10])
BBB_2014B Completed!
CCC_2014J Start...
ALLDataSet torch.Size([2255, 288, 10])
CCC_2014J Completed!
CCC_2014B Start...
ALLDataSet torch.Size([1644, 258, 10])
CCC_2014B Completed!
DDD_2013J Start...
ALLDataSet torch.Size([1748, 280, 10])
DDD_2013J Completed!
DDD_2014J Start...
ALLDataSet torch.Size([1618, 288, 10])
DDD_2014J Completed!
DDD_2013B Start...
ALLDataSet torch.Size([1199, 257, 10])
DDD_2013B Completed!
DDD_2014B Start...
ALLDataSet torch.Size([1104, 258, 10])
DDD_2014B Completed!
EEE_2013J Start...
ALLDataSet torch.Size([951, 280, 10